In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Baca daftar URL dari file CSV
urls = pd.read_csv("weather_urls.csv", header=None)[0].tolist()

all_data = []

for BASE_URL in urls:
    print(f"🔹 Ambil data dari: {BASE_URL}")
    response = requests.get(BASE_URL)
    soup = BeautifulSoup(response.text, "html.parser")

    # Ambil semua tanggal yang tersedia di dropdown
    date_options = soup.select("#wt-his-select option")
    dates = [opt["value"] for opt in date_options]

    for date_code in dates:
        print(f"   📅 Scraping data tanggal {date_code} ...")
        url = f"{BASE_URL}&hd={date_code}"
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")

        table = soup.find("table", {"id": "wt-his"})
        if not table:
            print(f"   ⚠️ Tabel tidak ditemukan untuk {date_code}")
            continue

        rows = table.find("tbody").find_all("tr")
        for row in rows:
            th = row.find("th")
            tds = row.find_all("td")

            if not th or len(tds) < 8:
                continue

            time_str = th.text.strip().replace("\n", " ")
            temp = tds[1].text.strip()
            weather = tds[2].text.strip()
            wind_speed = tds[3].text.strip()
            wind_dir = tds[4].find("span")["title"] if tds[4].find("span") else ""
            humidity = tds[5].text.strip()
            barometer = tds[6].text.strip()
            visibility = tds[7].text.strip()

            all_data.append({
                "Date": date_code,
                "Time": time_str,
                "Temp": temp,
                "Weather": weather,
                "Wind_Speed": wind_speed,
                "Wind_Direction": wind_dir,
                "Humidity": humidity,
                "Barometer": barometer,
                "Visibility": visibility
            })

        time.sleep(2)  # jeda biar aman

# Simpan hasil ke CSV
df = pd.DataFrame(all_data)
df.to_csv("weather_jakarta_Jan_to_Oct_2025.csv", index=False, encoding="utf-8-sig")

print("✅ Semua data dari Januari–Oktober 2025 sudah disimpan ke 'weather_jakarta_Jan_to_Oct_2025.csv'")